In [1]:
import numpy as np
import tensorflow as tf 


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,Flatten, Conv2D
from keras import regularizers
import scipy.io
from keras import optimizers
from keras.utils import np_utils
from scipy import signal
import mat73
from keras.callbacks import EarlyStopping
from utils.metrics import performance_measure

In [2]:
resample_rate = 100

mat1 = scipy.io.loadmat('../Data/Train_A.mat')

X_train = np.array(mat1['Epoch_train_A']) 
X_train = signal.resample(X_train, resample_rate, t=None, axis=1)
y_train = np.array(mat1['Y_train_A']) 
del mat1

In [ ]:
test_dataset = mat73.loadmat('../Data/Test_A.mat')
X_test = np.array(test_dataset['Epoch_test_A']) 
X_test = signal.resample(X_test, resample_rate, t=None, axis=1)
X_test = np.transpose(X_test , [0 , 2 , 1])
X_test = scipy.stats.zscore(X_test, axis=1)
event = np.array(test_dataset['event'])
test_str_A = test_dataset['test_str_A']
del test_dataset

In [ ]:
n_samples = X_train.shape[0]
length = X_train.shape[1]
n_channels = X_train.shape[2]

X_train = np.transpose(X_train , [0 , 2 , 1])
X_train = np.reshape(X_train , [n_samples , n_channels , length , 1])
X_train = scipy.stats.zscore(X_train, axis=1)
y_train = np_utils.to_categorical(y_train, 2)

In [ ]:
model = Sequential()
model.add (Conv2D(16 , (1, n_channels) , strides=1 ,  padding='same' , activation = 'relu' , input_shape=(n_channels , length , 1), 
                  kernel_regularizer=regularizers.L2(0.005)))

model.add (Conv2D(16 , (int(length/5), 1) , strides=int(length/5) ,  padding='same' , activation = 'relu' ,
                  kernel_regularizer=regularizers.L2(0.005)))
model.add (Flatten())
model.add(Dense(100, activation ='relu', kernel_regularizer=regularizers.L2(0.005)))
model.add( Dropout(0.25))
model.add(Dense(16, activation ='relu', kernel_regularizer=regularizers.L2(0.005)))  # W_regularizer= regularizers.l2(0.01)
model.add( Dropout(0.25))
model.add(Dense(2 , activation = 'softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam' , metrics=['accuracy'])

class_weights={0:1.,1:5.}
early_stopping_monitor = EarlyStopping(patience=8, restore_best_weights=True)
history = model.fit(X_train, y_train , batch_size=128 , epochs = 20 , callbacks= [early_stopping_monitor] , validation_split =0.2 , shuffle=True, class_weight=class_weights)

In [12]:
prediction = model.predict(X_test, batch_size = 128)
accuracy, precision, recall, f1, support = performance_measure(prediction, test_str_A, event)
print('acc_test: {}, precision: {}, recall: {}, f1:{}'.format(accuracy, precision, recall, f1))

acc_test: 0.95, precision: 0.8833333333333333, recall: 0.8878787878787879, f1:0.8795794681508967


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
